In [1]:
from dotenv import load_dotenv
import os

# Make sure to go to your service account and add BigQuery User role
load_dotenv('../.env')
# load_dotenv()
credential_path = os.getenv("GCP_SERVICE_ACCOUNT_KEY") 
gcp_project_id = os.getenv("GCP_PROJECT_ID")

In [2]:
# Creating an engine
from sqlalchemy import create_engine
from sqlalchemy import MetaData, Table
from sqlalchemy import select, and_

engine = create_engine(f"bigquery://{gcp_project_id}",
                       credentials_path=credential_path)

In [3]:
project_name = 'bigquery-public-data'
dataset_name = 'san_francisco_311'
table_name = '311_service_requests'  

# Connect to a table and loads its schema and other metadata
sf_table = Table(
        table_name,
        MetaData(), # added empty MetaData object
        autoload_with=engine,
        schema=f"{project_name}.{dataset_name}")


E0000 00:00:1759185200.867311   18296 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [4]:
sf_table

Table('311_service_requests', MetaData(), Column('unique_key', Integer(), table=<311_service_requests>), Column('created_date', TIMESTAMP(), table=<311_service_requests>), Column('closed_date', TIMESTAMP(), table=<311_service_requests>), Column('resolution_action_updated_date', TIMESTAMP(), table=<311_service_requests>), Column('status', String(), table=<311_service_requests>), Column('status_notes', String(), table=<311_service_requests>), Column('agency_name', String(), table=<311_service_requests>), Column('category', String(), table=<311_service_requests>), Column('complaint_type', String(), table=<311_service_requests>), Column('descriptor', String(), table=<311_service_requests>), Column('incident_address', String(), table=<311_service_requests>), Column('supervisor_district', Integer(), table=<311_service_requests>), Column('neighborhood', String(), table=<311_service_requests>), Column('location', String(), table=<311_service_requests>), Column('source', String(), table=<311_se

In [5]:
query = select(sf_table.c.unique_key, sf_table.c.category, sf_table.c.created_date)\
        .where(and_(sf_table.c.status == "Open", sf_table.c.complaint_type.contains("garbage")))\
        .order_by(sf_table.c.created_date.desc())\
        .limit(10)


# query

In [6]:
with engine.connect() as connection:
    print("\nQuery results:")
    result = connection.execute(query)
    for row in result:
        print(row)
    connection.commit() # only needed for INSERT, UPDATE, DELETE


Query results:
(101002686986, 'Street and Sidewalk Cleaning', datetime.datetime(2025, 9, 27, 12, 59, 41, tzinfo=datetime.timezone.utc))
(101002686977, 'Street and Sidewalk Cleaning', datetime.datetime(2025, 9, 27, 12, 58, 52, tzinfo=datetime.timezone.utc))
(101002686979, 'Street and Sidewalk Cleaning', datetime.datetime(2025, 9, 27, 12, 58, 50, tzinfo=datetime.timezone.utc))
(101002686956, 'Street and Sidewalk Cleaning', datetime.datetime(2025, 9, 27, 12, 54, 48, tzinfo=datetime.timezone.utc))
(101002686951, 'Street and Sidewalk Cleaning', datetime.datetime(2025, 9, 27, 12, 53, 54, tzinfo=datetime.timezone.utc))
(101002686948, 'Street and Sidewalk Cleaning', datetime.datetime(2025, 9, 27, 12, 53, 17, tzinfo=datetime.timezone.utc))
(101002686946, 'Street and Sidewalk Cleaning', datetime.datetime(2025, 9, 27, 12, 52, 47, tzinfo=datetime.timezone.utc))
(101002686943, 'Street and Sidewalk Cleaning', datetime.datetime(2025, 9, 27, 12, 51, 55, tzinfo=datetime.timezone.utc))
(101002686942, '